In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers
from keras.callbacks import ModelCheckpoint

import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)




# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val=x_val.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_val=(x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)


x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

mc = ModelCheckpoint('best_and_fast_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=150,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule),mc])

#save to disk
model_json = model.to_json()
with open('best_and_fast_model.json', 'w') as json_file:
    json_file.write(model_json)



Epoch 1/150
625/625 [==============================] - 16s 25ms/step - loss: 1.9997 - acc: 0.4037 - val_loss: 1.4368 - val_acc: 0.5559

Epoch 00001: val_acc improved from -inf to 0.55590, saving model to best_and_fast_model.h5
Epoch 2/150
625/625 [==============================] - 14s 22ms/step - loss: 1.3591 - acc: 0.5652 - val_loss: 1.3548 - val_acc: 0.6028

Epoch 00002: val_acc improved from 0.55590 to 0.60280, saving model to best_and_fast_model.h5
Epoch 3/150
625/625 [==============================] - 14s 22ms/step - loss: 1.1459 - acc: 0.6356 - val_loss: 0.9450 - val_acc: 0.7111

Epoch 00003: val_acc improved from 0.60280 to 0.71110, saving model to best_and_fast_model.h5
Epoch 4/150
625/625 [==============================] - 14s 22ms/step - loss: 1.0287 - acc: 0.6714 - val_loss: 0.8950 - val_acc: 0.7180

Epoch 00004: val_acc improved from 0.71110 to 0.71800, saving model to best_and_fast_model.h5
Epoch 5/150
625/625 [==============================] - 14s 22ms/step - loss: 0.9586

625/625 [==============================] - 19s 31ms/step - loss: 0.6131 - acc: 0.8407 - val_loss: 0.5778 - val_acc: 0.8538

Epoch 00041: val_acc improved from 0.85140 to 0.85380, saving model to best_and_fast_model.h5
Epoch 42/150
625/625 [==============================] - 19s 31ms/step - loss: 0.6161 - acc: 0.8391 - val_loss: 0.5754 - val_acc: 0.8560

Epoch 00042: val_acc improved from 0.85380 to 0.85600, saving model to best_and_fast_model.h5
Epoch 43/150
625/625 [==============================] - 19s 31ms/step - loss: 0.6118 - acc: 0.8411 - val_loss: 0.5902 - val_acc: 0.8532

Epoch 00043: val_acc did not improve from 0.85600
Epoch 44/150
625/625 [==============================] - 19s 31ms/step - loss: 0.6093 - acc: 0.8401 - val_loss: 0.5931 - val_acc: 0.8513

Epoch 00044: val_acc did not improve from 0.85600
Epoch 45/150
625/625 [==============================] - 19s 31ms/step - loss: 0.6129 - acc: 0.8403 - val_loss: 0.6464 - val_acc: 0.8395

Epoch 00045: val_acc did not improve fro

625/625 [==============================] - 19s 31ms/step - loss: 0.4861 - acc: 0.8795 - val_loss: 0.5129 - val_acc: 0.8793

Epoch 00083: val_acc did not improve from 0.88640
Epoch 84/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4834 - acc: 0.8812 - val_loss: 0.4813 - val_acc: 0.8853

Epoch 00084: val_acc did not improve from 0.88640
Epoch 85/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4879 - acc: 0.8792 - val_loss: 0.5753 - val_acc: 0.8595

Epoch 00085: val_acc did not improve from 0.88640
Epoch 86/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4781 - acc: 0.8797 - val_loss: 0.4740 - val_acc: 0.8856

Epoch 00086: val_acc did not improve from 0.88640
Epoch 87/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4777 - acc: 0.8812 - val_loss: 0.5136 - val_acc: 0.8776

Epoch 00087: val_acc did not improve from 0.88640
Epoch 88/150
625/625 [==============================] - 19s 31ms/step - loss: 

625/625 [==============================] - 19s 30ms/step - loss: 0.4374 - acc: 0.8885 - val_loss: 0.5142 - val_acc: 0.8746

Epoch 00126: val_acc did not improve from 0.89610
Epoch 127/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4349 - acc: 0.8889 - val_loss: 0.4942 - val_acc: 0.8768

Epoch 00127: val_acc did not improve from 0.89610
Epoch 128/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4415 - acc: 0.8864 - val_loss: 0.4563 - val_acc: 0.8862

Epoch 00128: val_acc did not improve from 0.89610
Epoch 129/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4331 - acc: 0.8885 - val_loss: 0.4812 - val_acc: 0.8829

Epoch 00129: val_acc did not improve from 0.89610
Epoch 130/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4349 - acc: 0.8890 - val_loss: 0.4950 - val_acc: 0.8794

Epoch 00130: val_acc did not improve from 0.89610
Epoch 131/150
625/625 [==============================] - 19s 31ms/step - l

In [7]:
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


10000/10000 [==============================] - 2s 152us/step

Test result: 87.770 loss: 0.492


In [8]:
model.save('best_and_fast_final.h5')


In [9]:
del model
from keras.models import load_model
model= load_model('best_and_fast_model.h5')
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 2s 168us/step

Test result: 88.410 loss: 0.464
